# Анализ работы API (CPU vs GPU)

In [1]:
import os
import requests
from sentence_splitter import SentenceSplitter, split_text_into_sentences
import time

## Множество одновременных обращений к API

In [2]:
def en_ru(title='************'):
    sourceid="en"
    targetid="ru"
    source = f'input.{sourceid}'
    target = f'api-output.en-{targetid}'
    url = 'http://10.239.16.89:60001/api/translation' 
    headers = {'accept': 'application/json',
               'Content-Type': 'application/json',
    }
    with open(source, encoding='utf-8') as f:
        text = f.readlines()
    text = (" ".join([i.strip() for i in text]))
    text = split_text_into_sentences(text, language=sourceid)

    result = []
    n = 0
    all_time = 0
    for sent in text:
        start = time.time() # начало замера времени
        n += 1
        sent = sent.replace('"', '\\"').encode('ascii', errors='ignore').decode("utf-8")
        data = '{ \"text\": \"'+sent+'\", \"sourceLanguage\": \"'+sourceid+'\", \"targetLanguage\": \"'+targetid+'\"}'
        answer = requests.post(url, headers=headers, data=data).json()['message']        
        end = time.time() # окончание замера времени
        all_time += (end - start) # фиксация затраченного времени
        result.append(answer)

#     print('EN-RU\n', title)
#     print(f'Затрачено времени: {all_time:.2f} сек.')
    return all_time


In [3]:
def ar_ru(title='************'):
    sourceid="ar"
    targetid="ru"
    source = f'input.{sourceid}'
    target = f'api-output.en-{targetid}'
    url = 'http://10.239.16.89:60001/api/translation' 
    headers = {'accept': 'application/json',
               'Content-Type': 'application/json',
    }

    result = []
    all_time = 0
    with open(source, encoding='utf-8') as f:
        n = 0
        for i in f:
            start = time.time() # начало замера времени
            sent = i.strip()
            n += 1
            sent = sent.encode('utf-8').decode('latin-1', 'ignore')
            data = '{\n  "text": "' + str(sent) + '",\n  "sourceLanguage": "' + sourceid + '",\n  "targetLanguage": "' + targetid + '"\n}'
            answer = requests.post(url, headers=headers, data=data).json()['message']            

            end = time.time() # окончание замера времени
            all_time += (end - start) # фиксация затраченного времени
            result.append(answer)

#     print('AR-RU\n', title)
#     print(f'Затрачено времени: {all_time:.2f} сек.')
    return all_time

In [23]:
title = '20 CPU'
s = 0
s += en_ru(title)
s += ar_ru(title)
print(title)
print(f'Затрачено времени: {s:.2f} сек.')

20 CPU
Затрачено времени: 88.34 сек.


In [9]:
def full(title='20 CPU'):
    s = 0
    s += en_ru(title)
    s += ar_ru(title)
    print(title)
    print(f'Затрачено времени: {s:.2f} сек.')

In [26]:
full()

20 CPU
Затрачено времени: 88.46 сек.


### Многопоточность

In [10]:
from concurrent.futures import ThreadPoolExecutor

def run_concurrent_burgers():
    
    # Выполнить три задачи, используя пул потоков
    with ThreadPoolExecutor() as executor:
        executor.submit(full)
        executor.submit(full)
        executor.submit(full)

In [32]:
run_concurrent_burgers()

20 CPU
Затрачено времени: 241.75 сек.
20 CPU
Затрачено времени: 241.84 сек.
20 CPU
Затрачено времени: 241.86 сек.


In [39]:
run_concurrent_burgers()

C:\Users\E64B5~1.BOL\AppData\Local\Temp/ipykernel_3760/535178405.py:7: RuntimeWarning: coroutine 'full' was never awaited
  executor.submit(full)
C:\Users\E64B5~1.BOL\AppData\Local\Temp/ipykernel_3760/535178405.py:9: RuntimeWarning: coroutine 'full' was never awaited
  executor.submit(full)


### Асинхронность

In [35]:
import asyncio

async def run_concurrent_burgers():
    await asyncio.gather(
        full(),
        full(),
        full(),
    )

In [38]:
try:
    await run_concurrent_burgers()
except TypeError:
    pass

20 CPU
Затрачено времени: 89.58 сек.
20 CPU
Затрачено времени: 89.81 сек.
20 CPU
Затрачено времени: 89.07 сек.


### Многопроцессорность

In [42]:
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor

In [46]:
result = []

In [47]:
def full(title='20 CPU'):
    s = 0
    s += en_ru(title)
    s += ar_ru(title)
    result.append(s)

In [48]:
def run_parallel_salads():
    result = 0
    with ProcessPoolExecutor(16) as executor:
        executor.submit(
            full(),
            full(),
            full(),
        )

In [49]:
run_parallel_salads(); print(sum(result) / len(result))

[90.03459143638611, 90.22947382926941, 88.21230292320251]


In [50]:
"full()," * 5

'full(),full(),full(),full(),full(),'

# Цикл

In [ ]:
from concurrent.futures import ProcessPoolExecutor

total = {}
for i in range(1, 20):
    result = []
    
    def full(title='20 CPU'):
        s = 0
        s += en_ru(title)
        s += ar_ru(title)
        result.append(s)
    
    cmd = "full()," * i
    
    start = time.time() # начало замера времени
    def main():    
        with ProcessPoolExecutor(16) as executor:
            executor.submit(eval(cmd))
    main()
    end = time.time() # окончание замера времени
    
    unit = sum(result) / len(result)
    func = (end - start) / i
    total[i] = [unit, func]
    
    print(i, unit, func)

1 89.58155274391174 89.71497964859009
2 90.31866896152496 90.38049459457397
3 90.21124426523845 90.25512250264485
4 90.6158657670021 90.6448403596878
5 89.987788438797 90.01864337921143
6 90.05073471864064 90.08987144629161
7 89.80033278465271 89.828122649874
8 89.84777408838272 89.8721191585064
